In [ ]:
!pip install openai transformers accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.3 MB/s eta 0:00:00


In [ ]:
import os
from transformers import pipeline
import openai

In [ ]:
use_openai = False

openai.api_key = "sk-..."

unsafe_keywords = ["cure cancer", "dosage", "overdose", "emergency", "prescribe", "treatment for"]

In [ ]:
unsafe_keywords = ["cure cancer", "dosage", "overdose", "emergency", "prescribe", "treatment for"]

In [ ]:
def is_safe_query(user_input):
    for word in unsafe_keywords:
        if word.lower() in user_input.lower():
            return False
    return True

In [ ]:
!pip install transformers accelerate --quiet

from transformers import pipeline

generator = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", device_map="auto")

def format_hf_prompt(user_input):
    return f"""### Instruction:
You are a helpful and friendly medical assistant.
Only provide general, factual, and safe health information.
Do NOT give any medical diagnosis, treatment, dosage, or emergency advice.
Always advise the user to consult a doctor.

### User:
{user_input}

### Assistant:
"""

def get_health_response(user_input):
    prompt = format_hf_prompt(user_input)
    result = generator(prompt, max_new_tokens=200, temperature=0.5)[0]['generated_text']
    reply = result.split("### Assistant:")[-1].strip()
    return reply

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
def format_openai_prompt(user_input):
    return f"""You are a helpful and friendly medical assistant.
Your goal is to provide general, factual, and safe health information.
Do NOT give any diagnosis, treatment, dosage, or emergency advice.
Always advise users to consult a doctor for medical decisions.

User: {user_input}
Assistant:"""

def format_hf_prompt(user_input):
    return f"""### Instruction:
You are a helpful and friendly medical assistant.
Only provide general, factual, and safe health information.
Do NOT give any medical diagnosis, treatment, dosage, or emergency advice.
Always advise the user to consult a doctor.

### User:
{user_input}

### Assistant:
"""

if not use_openai:
    from transformers import pipeline
    generator = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", device_map="auto")

def get_health_response(user_input):
    if not is_safe_query(user_input):
        return "I'm sorry, I can't provide medical treatment or emergency advice. Please consult a doctor."
    if use_openai:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful and friendly medical assistant."},
                    {"role": "user", "content": user_input}
                ],
                max_tokens=300,
                temperature=0.5
            )
            return response['choices'][0]['message']['content'].strip()
        except Exception as e:
            return f"An error occurred: {e}"
    else:
        try:
            prompt = format_hf_prompt(user_input)
            result = generator(prompt, max_new_tokens=200, temperature=0.5)[0]['generated_text']
            reply = result.split("### Assistant:")[-1].strip()
            return reply
        except Exception as e:
            return f"An error occurred: {e}"


In [ ]:
generator = pipeline("text-generation", model="distilgpt2", device_map="auto")
generator = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", device=0)
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

def get_health_response(user_input):
    if not is_safe_query(user_input):
        return "I'm sorry, I can't provide medical treatment or emergency advice. Please consult a doctor."
    input_ids = tokenizer.encode(user_input, return_tensors="pt")
    with torch.no_grad():
        output = model.generate(input_ids, max_length=100, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response[len(user_input):].strip()


In [ ]:

def run_chatbot():
    print("🤖 HealthBot: Hello! I can answer general health questions. (Type 'exit' to quit)\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("HealthBot: Take care! Goodbye.")
            break
        reply = get_health_response(user_input)
        print(f"HealthBot: {reply}\n")

run_chatbot()

🤖 HealthBot: Hello! I can answer general health questions. (Type 'exit' to quit)

You: quit
HealthBot: Take care! Goodbye.
